In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Customers
    ''',
    con,
)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023,Mexico
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden
...,...,...,...,...,...,...,...
86,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland
87,88,Wellington Importadora,Paula Parente,"Rua do Mercado, 12",Resende,08737-363,Brazil
88,89,White Clover Markets,Karl Jablonski,305 - 14th Ave. S. Suite 3B,Seattle,98128,USA
89,90,Wilman Kala,Matti Karttunen,Keskuskatu 45,Helsinki,21240,Finland


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address FROM Customers
    WHERE Country = 'Germany' 
    OR Country = 'France' 
    OR Country = 'Spain'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, COUNT(Country) cnt FROM Customers 
    GROUP BY Country ORDER BY cnt DESC limit(3)
    ''',
    con,
)

,Country,cnt
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT t.OrderDate,
    sh.ShipperName
    FROM(
    WITH tab AS (
    SELECT *,
    row_number() over() number,
    SUBSTR(OrderDate, 1, 2) days,
    SUBSTR(OrderDate, 4, 2) month,
    SUBSTR(OrderDate, 7, 4) year,
    ShipperID
    FROM Orders
    ORDER BY year, month, days)
    SELECT * FROM tab
    WHERE number = 10) t
    LEFT JOIN Shippers sh
    ON sh.ShipperID = t.ShipperID
    ''',
    con,
)

,OrderDate,ShipperName
0,16.07.1996,Federal Shipping


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [10]:
tasks+=1
pd.read_sql(
    '''
    SELECT OrderDetails.*, 
    Products.Price, 
    Products.ProductName
    FROM OrderDetails
    LEFT JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    WHERE OrderID = 
        (SELECT OrderID FROM
    (SELECT OrderID, SUM(multi) totalprice FROM 
    (SELECT *,
    Quantity*Price multi
    FROM (SELECT OrderDetails.*, 
    Products.Price
    FROM OrderDetails
    LEFT JOIN Products
    ON OrderDetails.ProductID = Products.ProductID) t) tt
    GROUP BY OrderID 
    ORDER BY totalprice DESC LIMIT(1)))
    ''',
    con,
)

,OrderDetailID,OrderID,ProductID,Quantity,Price,ProductName
0,331,10372,20,12,81.0,Sir Rodney's Marmalade
1,332,10372,38,40,263.5,Côte de Blaye
2,333,10372,60,70,34.0,Camembert Pierrot
3,334,10372,72,42,34.8,Mozzarella di Giovanni


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT SUM(Quantity) AS MAXQ, OrderID,
    Products.ProductName
    FROM OrderDetails
    LEFT JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    GROUP BY Products.ProductID
    ORDER BY MAXQ DESC LIMIT(1)
    ''',
    con,
)

,MAXQ,OrderID,ProductName
0,458,10253,Gorgonzola Telino


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT COUNT(OrderID) AS NUMBER, Suppliers.SupplierName, Suppliers.Country, 
    Suppliers.ContactName, Suppliers.Phone
    FROM OrderDetails
    LEFT JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    LEFT JOIN Suppliers
    ON Products.SupplierID = Suppliers.SupplierID
    GROUP BY Suppliers.SupplierName
    ORDER BY NUMBER DESC LIMIT(5)   
    ''',
    con,
)

,NUMBER,SupplierName,Country,ContactName,Phone
0,42,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,40,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,33,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,32,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,31,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, SUM(OrderDetails.Quantity*Products.Price) AS Cost, Categories.CategoryName
    FROM Customers
    LEFT JOIN Orders
    ON Orders.CustomerID = Customers.CustomerID
    LEFT JOIN OrderDetails
    ON OrderDetails.OrderID = Orders.OrderID
    LEFT JOIN Products
    ON Products.ProductID = OrderDetails.ProductID
    LEFT JOIN Categories
    ON Categories.CategoryID = Products.CategoryID
    WHERE Country = 'Brazil'
    GROUP BY Products.CategoryID
    ORDER BY Cost DESC LIMIT(1)
    ''',
    con,
)

,Country,Cost,CategoryName
0,Brazil,13690.0,Beverages


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT ((SELECT SUM(OrderDetails.Quantity*Products.Price) AS COST
    FROM Customers
    LEFT JOIN Orders
    ON Orders.CustomerID = Customers.CustomerID
    LEFT JOIN OrderDetails
    ON OrderDetails.OrderID = Orders.OrderID
    LEFT JOIN Products
    ON Products.ProductID = OrderDetails.ProductID
    WHERE Country = 'USA'
    GROUP BY OrderDetails.OrderID
    ORDER BY COST DESC LIMIT(1)) -
    (SELECT SUM(OrderDetails.Quantity*Products.Price) AS COST
    FROM Customers
    LEFT JOIN Orders
    ON Orders.CustomerID = Customers.CustomerID
    LEFT JOIN OrderDetails
    ON OrderDetails.OrderID = Orders.OrderID
    LEFT JOIN Products
    ON Products.ProductID = OrderDetails.ProductID
    WHERE Country = 'USA'
    GROUP BY OrderDetails.OrderID
    ORDER BY COST ASC LIMIT 1,1)) AS Difference
    ''',
    con,
)

,Difference
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [15]:
tasks+=1
pd.read_sql(
    '''
    SELECT Count(Orders.EmployeeID) AS Quantity, (FirstName || " " || LastName) AS Employee
    FROM Employees
    LEFT JOIN Orders
    ON Orders.EmployeeID = Employees.EmployeeID
    WHERE FirstName = 'Anne' or FirstName = 'Nancy' or FirstName = 'Janet'
    GROUP BY Orders.EmployeeID
    ORDER BY Quantity DESC
    ''',
    con,
)

,Quantity,Employee
0,31,Janet Leverling
1,29,Nancy Davolio
2,6,Anne Dodsworth


10. Сколько банок крабового мяса всего было заказано.

In [16]:
tasks+=1
pd.read_sql(
    '''
    SELECT SUM(Quantity) AS Quantity
    FROM OrderDetails
    LEFT JOIN Products
    ON Products.ProductID = OrderDetails.ProductID
    WHERE Products.ProductName = 'Boston Crab Meat'
    ''',
    con,
)

,Quantity
0,256


In [17]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
